## 1. INITIATION & READING DATA

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', -1)

import seaborn as sns
sns.set_style('darkgrid')

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoLars, ElasticNet, RidgeCV, LassoCV, LassoLarsCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, SCORERS
from sklearn.model_selection import train_test_split, cross_val_score


print('** HOUSE PRICES - ADVANCED REGRESSION TECHNIQUES **')
print("-"*25)

import gc
import os
for x in os.listdir("input/"):
    print(x)
print("-"*25)

print ("Initiation is DONE !!")
print("-"*25)

In [ ]:
##### SOME USEFULL FUNCTIONS

## TIMER
import time
from contextlib import contextmanager
@contextmanager
def timer():
    t0 = time.time()
    yield
    t1 = time.time()
    print( 'Runtime: ' + str(round(t1-t0,2)) + ' sn' )
    
    
def null_detector(x):
    missing = x.isnull().sum()
    missing = missing[missing > 0]
    missing.sort_values(ascending=False, inplace=True)
    
    fig, ax = plt.subplots(figsize=(15,6) )
    sns.barplot( x=missing.index, y=missing.values)
    plt.title( f'Total Column: {x.shape[1]} --- Missing Column: {len(missing)} ({round(len(missing)*100/x.shape[1],2)}%) ' )
    plt.xticks(rotation=60)
    
    for p in ax.patches:
        ax.annotate( '{:.2f}'.format(p.get_height()/x.shape[0]),  (p.get_x() , p.get_height() + 3), fontsize=11 )  
    plt.show()
    
    
def target_dist_visualizer(y):
    import scipy.stats as st
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,4) )

    sns.distplot(y, kde=True, fit=st.norm, ax=ax1)
    ax1.set(title='Normal')

    sns.distplot(y, kde=True, fit=st.lognorm, ax=ax2)
    ax2.set(title='Log Normal')

    sns.distplot(y, kde=True, fit=st.johnsonsu, ax=ax3)
    ax3.set(title='Johnson SU')
    plt.show()
    print('-'*120)
    
    df_res = pd.DataFrame({ 'normaltest-stat': [ st.normaltest(y)[0]  ],
                            'normaltest-p'   : [ st.normaltest(y)[1]  ],
                            'shapiro-stat'   : [ st.shapiro(y)[0]     ],
                            'shapiro-p'      : [ st.shapiro(y)[1]     ],
                            'anderson-stat'  : [ st.anderson(y, dist='norm')[0] ],
                            'anderson-crit'  : [ st.anderson(y, dist='norm')[1]  ],
                            'anderson-sgnf'  : [ st.anderson(y, dist='norm')[2]  ]   })
    
    return df_res

    
def num_col_visualizer(train, test, num_cols, target, box_limit=10):
    i=0
    for col in num_cols:
        i += 1
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5) )
        # plt.suptitle(col, fontsize=20)

        ## FIRST SUBPLOT
        if train[col].nunique() <= box_limit:
            sns.boxplot(data=train,  x=col, y=target, ax=ax1)
        else:
            sns.scatterplot( data=train,  x=col, y=target, ax=ax1)

        ax1.set( title = str(i) + '. ' + col + ' vs. ' + target ) 

        ## SECOND SUBPLOT
        try:
            sns.distplot(train[col], kde=True, ax=ax2)
        except RuntimeError as re:
            if str(re).startswith("Selected KDE bandwidth is 0. Cannot estiamte density."):
                sns.distplot(train[col], kde=True, kde_kws={'bw': 0.1}, ax=ax2)
            else:
                raise re
        ax2.set( title = 'Train --- Null: ' + str(train[col].isnull().sum()) + ' --- Unique: ' + str( train[col].nunique()  ) )

        ## THIRD SUBPLOT
        try:
            sns.distplot(test[col], kde=True, ax=ax3)
        except RuntimeError as re:
            if str(re).startswith("Selected KDE bandwidth is 0. Cannot estiamte density."):
                sns.distplot(test[col], kde=True, kde_kws={'bw': 0.1}, ax=ax3)
            else:
                raise re
        ax3.set( title = 'Test --- Null: ' + str(test[col].isnull().sum())  + ' --- Unique: ' + str( test[col].nunique() ) )

        plt.show()
        print(train[col].describe(include='all'))
        print('-'*120)    


def str_col_visualizer(train, test, str_cols, target, box_limit=20):
    i = 0
    for col in str_cols:
        i+=1
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5) )
        # plt.suptitle(col, fontsize=20)

        ## FIRST SUBPLOT
        sns.boxplot(x=train[col].fillna('NaN'), y=train[target], ax=ax1)
        ax1.set( title = str(i) + '. ' + col + ' vs. ' + target ) 

        ## SECOND SUBPLOT
        h = train[col].value_counts()
        sns.barplot(x=h.index, y=h.values, ax=ax2)
        ax2.set( title = 'Train --- Null: ' + str(train[col].isnull().sum()) + ' --- Unique: ' + str( train[col].nunique()  ) )
        
        ## THIRD SUBPLOT
        g = test[col].value_counts()
        sns.barplot(x=g.index, y=g.values, ax=ax3)
        ax3.set( title = 'Test --- Null: ' + str(test[col].isnull().sum())  + ' --- Unique: ' + str( test[col].nunique() ) )
        
        plt.show()
        print(train[col].describe(include='all'))
        print('-'*120)  

def reg_with_one_exploratory_visualizer(df_train, exp_var, ind_var):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5) )
    # plt.suptitle(col, fontsize=20)

    ## FIRST SUBPLOT
    sns.regplot(x=df_train[exp_var], y=df_train[ind_var], ax=ax1)

    ## SECOND SUBPLOT
    sns.residplot(x=df_train[exp_var], y=df_train[ind_var], ax=ax2)

    plt.show()
    
def reg_result_visualizer(pred, y):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6) )
    
    sns.scatterplot(pred, pred-y, ax=ax1)
    ax1.set(title='Constant Variance')

    sns.distplot(pred-y, kde=True, fit=st.norm, ax=ax2)
    ax2.set(title='Normal Distribution')
    
    sns.scatterplot(range(len(y)), pred-y, ax=ax3)
    ax3.set(title='No Autocorrelation')
    
    plt.show()

    return

result = pd.DataFrame()

In [ ]:
path = 'input/'
out_path = 'output/'

with timer():
    train      = pd.read_csv( f'{path}train.csv')
    test       = pd.read_csv( f'{path}test.csv')
    submission = pd.read_csv( f'{path}sample_submission.csv')

print('Reading Data is DONE !!')
print("-"*25)

print( f'train      shape :  {train.shape}       ')
print( f'test       shape :  {test.shape}        ')
print( f'submission shape :  {submission.shape}  ')

In [ ]:
target   = 'SalePrice'
str_cols = [f for f in train.columns if train.dtypes[f] == 'object']
num_cols = [f for f in train.columns if train.dtypes[f] != 'object']
num_cols.remove(target)
num_cols.remove('Id')

print(len(str_cols))
print(len(num_cols))

In [ ]:
train.dtypes.value_counts()

## TREATMENT OF TARGET VARIABLES

In [ ]:
## This part will be improved with advanced methods
y = train[target]
target_dist_visualizer(y)

In [ ]:
## This part will be improved with advanced methods
y = np.log1p(train[target])
target_dist_visualizer(y)

In [ ]:
## p < 0.05 indicates the rejection of null hypotesis, 
## (H0: that is the series comes from a normal distribution)

In [ ]:
train[target] = np.log1p(train[target])

## NUMERICAL COLUMN VISUALISATION 

In [ ]:
print(train.shape)
train = train[ ~( train['GrLivArea']   >= 4000 ) ]
print(train.shape)
train = train[ ~( train['LotFrontage'] >= 200  ) ]
print(train.shape)

In [ ]:
df = pd.concat([train, test], axis=0, ignore_index=True )
print(df.shape)
df.tail()

In [ ]:
# num_col_visualizer(train, test, num_cols, target, box_limit=20)

In [ ]:
## Numerical Features Simple Selection and Handling
# Seems like ordinal but it is not
df['fea_MSSubClass']         = df['MSSubClass'].apply(lambda x:  3 if ( (x==60) | (x==120) ) else 1 if ( (x==30) | (x==45) | (x==180) ) else 2 )
df['fea_LotFrontage']        = df['LotFrontage'].fillna( df['LotFrontage'].mean() )
df['fea_LotFrontage_log']    = df['fea_LotFrontage'].apply(np.log1p)
df['fea_LotArea']            = df['LotArea']
df['fea_LotArea_log']        = df['LotArea'].apply(np.log1p)
df['fea_OverallQual']        = df['OverallQual']
df['fea_OverallCond']        = df['OverallCond']
df['fea_YearBuilt']          = df['YearBuilt']
df['fea_YearRemodAdd']       = df['YearRemodAdd']
df['fea_YrSold']             = df['YrSold']

df['fea_MasVnrArea']         = df['MasVnrArea'].fillna(0).apply(np.log1p)
df['fea_BsmtFinSF1']         = df['BsmtFinSF1'].fillna(0)
df['fea_BsmtFinSF2']         = df['BsmtFinSF2'].fillna(0)
df['fea_BsmtUnfSF']          = df['BsmtUnfSF'].fillna(0)
df['fea_BsmtUnfSF_log']      = df['BsmtUnfSF'].fillna(0).apply(np.log1p)
df['fea_TotalBsmtSF']        = df['TotalBsmtSF'].fillna( df['TotalBsmtSF'].mean() )
df['fea_TotalBsmtSF_log']    = df['TotalBsmtSF'].fillna( df['TotalBsmtSF'].mean() ).apply(np.log1p)


df['fea_1stFlrSF']           = df['1stFlrSF']
df['fea_1stFlrSF_log']       = df['1stFlrSF'].apply(np.log1p)
df['fea_2ndFlrSF']           = df['2ndFlrSF']
df['fea_2ndFlrSF_log']       = df['2ndFlrSF'].apply(np.log1p)
# df['fea_LowQualFinSF']       = df['LowQualFinSF']
df['fea_GrLivArea']          = df['GrLivArea']
df['fea_GrLivArea_log']      = df['GrLivArea'].apply(np.log1p)

df['fea_BsmtFullBath']       = df['BsmtFullBath'].fillna(0)
df['fea_BsmtHalfBath']       = df['BsmtHalfBath'].fillna(0)
df['fea_BsmtTotalBath']      = df['BsmtHalfBath'].fillna(0) + (df['BsmtHalfBath'].fillna(0) / 2)

df['fea_FullBath']           = df['FullBath']
df['fea_HalfBath']           = df['HalfBath']
df['fea_TotalBath']          = df['FullBath'].fillna(0) + (df['HalfBath'].fillna(0) / 2)

df['fea_BedroomAbvGr']       = df['BedroomAbvGr']
df['fea_KitchenAbvGr']       = df['KitchenAbvGr'] 
df['fea_TotRmsAbvGrd']       = df['TotRmsAbvGrd']

df['fea_Fireplaces']         = df['Fireplaces']
df['fea_GarageYrBlt']        = df['GarageYrBlt'].fillna( df['GarageYrBlt'].mean() )
df['fea_GarageCars']         = df['GarageCars'].fillna(0)

df['fea_GarageArea']         = df['GarageArea'].fillna(0)
df['fea_GarageArea_log']     = df['GarageArea'].apply(np.log1p)
df['fea_WoodDeckSF']         = df['WoodDeckSF']

df['fea_OpenPorchSF']        = df['OpenPorchSF']
df['fea_EnclosedPorch']      = df['EnclosedPorch']
df['fea_3SsnPorch']          = df['3SsnPorch']
df['fea_ScreenPorch']        = df['ScreenPorch']
df['fea_TotalPorch']         = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']

df['fea_PoolArea']           = df['PoolArea']
df['fea_MiscVal']            = df['MiscVal']

df['fea_MoSold']             = df['MoSold']
df['fea_YrSold']             = df['YrSold']

In [ ]:
# str_col_visualizer(train, test, str_cols, target, box_limit=12)

In [ ]:
# Encode some categorical features as ordered numbers when there is information in the order

In [ ]:
## String Features (Some nominal some ordinal, treat carefully !! )
df['fea_MSZoning']         = df['MSZoning'].fillna( df['MSZoning'].value_counts().index[0] )
df['fea_MSZoning']         = df['fea_MSZoning'].map( dict(df.groupby(['fea_MSZoning'])['SalePrice'].median().sort_values()) )

tmp = pd.get_dummies(df['Street'], prefix='fea_Street')
tmp.drop('fea_Street_Grvl', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

tmp = pd.get_dummies(df['Alley'], prefix='fea_Alley')
df = pd.concat( [df, tmp], axis=1 )

df['fea_LotShape']   =  df['LotShape'].map(  {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4} )

tmp = pd.get_dummies(df['LandContour'], prefix='fea_LandContour')
tmp.drop('fea_LandContour_Lvl', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

## Skip Utilities

# No Hope
tmp = pd.get_dummies(df['LotConfig'], prefix='fea_LotConfig')
tmp.drop('fea_LotConfig_Inside', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

# No Hope
df['fea_LandSlope']   =  df['LandSlope'].map( {"Sev" : 3, "Mod" : 2, "Gtl" : 1} )

df['fea_Neighborhood']  = df['Neighborhood'].apply(lambda x: 3 if ( (x=='NoRidge') | (x=='NridgHt') | (x=='StoneBr') ) else
                                                            1 if ( (x=='IDOTRR')  | (x=='MeadowV') | (x=='BrDale' ) ) else 2 )
#No Hope
tmp = pd.get_dummies(df['Condition1'], prefix='fea_Condition1')
tmp.drop('fea_Condition1_Norm', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

#No Hope
tmp = pd.get_dummies(df['Condition2'], prefix='fea_Condition2')
tmp.drop('fea_Condition2_Norm', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

#No Hope
tmp = pd.get_dummies(df['BldgType'], prefix='fea_BldgType')
tmp.drop('fea_BldgType_1Fam', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_HouseStyle']         = df['HouseStyle'].map( dict(df.groupby(['HouseStyle'])['SalePrice'].median().sort_values()) )

# Skip RoofStyle
# Skip RoofMatl

#No Hope
tmp = pd.get_dummies(df['Exterior1st'], prefix='fea_Exterior1st')
tmp.drop( ['fea_Exterior1st_ImStucc', 'fea_Exterior1st_Stone' ], axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

#No Hope
tmp = pd.get_dummies(df['Exterior2nd'], prefix='fea_Exterior2nd')
tmp.drop( ['fea_Exterior2nd_Other'], axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

tmp = pd.get_dummies(df['MasVnrType'], prefix='fea_MasVnrType')
tmp.drop('fea_MasVnrType_None', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )


df['fea_ExterQual']   =  df['ExterQual'].map(  {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5} )
df['fea_ExterCond']   =  df['ExterCond'].map(  {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5} )

tmp = pd.get_dummies(df['Foundation'], prefix='fea_Foundation')
tmp.drop('fea_Foundation_Stone', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_BsmtQual']   =  df['BsmtQual'].map(  {np.nan:-1, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5} )

df['fea_BsmtCond']   =  df['BsmtCond'].map( {np.nan:-1, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

df['fea_BsmtExposure']   =  df['BsmtExposure'].map(   {np.nan:-1, "No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3} )

df['fea_BsmtFinType1']   =  df['BsmtFinType1'].map(   {np.nan:-1, "Unf" : 0, "LwQ" : 1, "Rec" : 2, "BLQ" : 3, "ALQ" : 4, "GLQ" : 5} )
df['fea_BsmtFinType1_GLQ']   =  df['BsmtFinType1'].map(   {np.nan:-1, "Unf" : 0, "LwQ" : 0, "Rec" : 0, "BLQ" : 0, "ALQ" : 0, "GLQ" : 1} )

df['fea_BsmtFinType2']   =  df['BsmtFinType2'].map(   {np.nan:-1, "Unf" : 0, "LwQ" : 1, "Rec" : 2, "BLQ" : 3, "ALQ" : 4, "GLQ" : 5} )

#No Hope
tmp = pd.get_dummies(df['Heating'], prefix='fea_Heating')
tmp.drop(['fea_Heating_OthW', 'fea_Heating_Floor'], axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_HeatingQC']   =  df['HeatingQC'].map(   {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5} )

tmp = pd.get_dummies(df['CentralAir'], prefix='fea_CentralAir')
tmp.drop('fea_CentralAir_N', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_Electrical']   =  df['Electrical'].map(   {np.nan:5, "Mix" : 1, "FuseP" : 2, "FuseF" : 3, "FuseA" : 4, "SBrkr" : 5} )

df['fea_KitchenQual']   =  df['KitchenQual'].map( {np.nan:0, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

#No Hope
tmp = pd.get_dummies(df['Functional'], prefix='fea_Functional')
tmp.drop('fea_Functional_Typ', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_FireplaceQu']   =  df['FireplaceQu'].map( {np.nan:0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

tmp = pd.get_dummies(df['GarageType'], prefix='fea_GarageType')
df = pd.concat( [df, tmp], axis=1 )

df['fea_GarageFinish']   =  df['GarageFinish'].map( {np.nan:0, "Unf" : 1, "RFn" : 2, "Fin": 3}  )

df['fea_GarageQual']   =  df['GarageQual'].map( {np.nan:0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

df['fea_GarageCond']   =  df['GarageCond'].map( {np.nan:0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

df['fea_PavedDrive']   =  df['PavedDrive'].map( { "N" : 1, "P" : 2, "Y": 3}  )

df['fea_PoolQC']   =  df['PoolQC'].map( {np.nan:0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}  )

#Skip Fence

#No Hope
tmp = pd.get_dummies(df['MiscFeature'], prefix='fea_MiscFeature')
tmp.drop(['fea_MiscFeature_Shed', 'fea_MiscFeature_TenC'], axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

tmp = pd.get_dummies(df['SaleType'], prefix='fea_SaleType')
tmp.drop('fea_SaleType_WD', axis=1, inplace=True)
df = pd.concat( [df, tmp], axis=1 )

df['fea_SaleCondition']         = df['SaleCondition'].map( dict(df.groupby(['SaleCondition'])['SalePrice'].median().sort_values()) )

print(df.shape)
df.head()

In [ ]:
##### COMBINATIONS OF EXISTING FEATURES

### OVERALL QUALITIES 
# Overall quality of the house
df["fea_OverallGrade"] = df["fea_OverallQual"] * df["fea_OverallCond"]
# Overall quality of the garage
df["fea_GarageGrade"] = df["fea_GarageQual"] * df["fea_GarageCond"]
# Overall quality of the exterior
df["fea_ExterGrade"] = df["fea_ExterQual"] * df["fea_ExterCond"]

### OVERALL SCORES
# Overall kitchen score
df["fea_KitchenScore"] = df["fea_KitchenAbvGr"] * df["fea_KitchenQual"]
# Overall fireplace score
df["fea_FireplaceScore"] = df["fea_Fireplaces"] * df["fea_FireplaceQu"]
# Overall garage score
df["fea_GarageScore"] = df["fea_GarageArea"] * df["fea_GarageQual"]
# Overall pool score
df["fea_PoolScore"] = df["fea_PoolArea"] * df["fea_PoolQC"]

### TOTAL NUMBERS

# Total SF for house (incl. basement)
df["fea_AllSF"] = df["fea_GrLivArea"] + df["fea_TotalBsmtSF"]
df["fea_AllSF_log"] = df["fea_AllSF"].apply(np.log1p)

In [ ]:
fea_cols = [col for col in df.columns if col.startswith('fea_')]
print(len(fea_cols))
df = df[fea_cols + ['SalePrice']]
print(df.shape)
df.head()

In [ ]:
df_scaled = df.copy()
df_scaled[fea_cols] = MinMaxScaler().fit_transform(df_scaled[fea_cols])

In [ ]:
df_train = df_scaled[df_scaled['SalePrice'].notnull()]
print(df_train.shape)
df_train.head()

In [ ]:
spearman = df_train.corr()['SalePrice'].reset_index().sort_values('SalePrice', ascending=False)
pd.concat([spearman.head(), spearman.tail()])

# 1. BASIC REGRESSION WITH ONE EXPLORATORY VARIABLE

In [ ]:
exp_vars = ['fea_AllSF']

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

print("-"*25)
reg_with_one_exploratory_visualizer(df_train, exp_vars, target)
print("-"*25)

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
# In the residual by predicted plot, we see that the residuals are randomly scattered around the center line of zero, with no obvious non-random pattern.
# Residuals have constant variance

# Residuals are approximately normally distributed

# No autocorrelation!! Residuals are independent one another, if doubt apply Durbin Watson

# 2. REGRESSION WITH ALL FEATURES

In [ ]:
exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = df_train[fea_cols].columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df_train[fea_cols].values, i) 
                          for i in range(len(df_train[fea_cols].columns))]
vif_data

# 3. REGRESSION WITH FEATURES - BACKWARD ELIMINATION

In [ ]:
X = df_train[fea_cols]
y = df_train['SalePrice']

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break

selected_features_BE = cols
print(selected_features_BE)
print(len(selected_features_BE))

In [ ]:
exp_vars = selected_features_BE.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = df_train[selected_features_BE].columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df_train[selected_features_BE].values, i) for i in range(len(df_train[selected_features_BE].columns))]
vif_data

In [ ]:
fig = plt.figure(figsize=(20,16) )
sns.heatmap( df_train[selected_features_BE].corr(),  annot=True  )
plt.title('xx')
plt.show()

# 4. REGRESSION WITH FEATURES DETERMINED BY CORRELATION

In [ ]:
spearman = df_train.corr()['SalePrice'].reset_index().sort_values('SalePrice', ascending=False)
corred  = spearman[ (spearman['SalePrice'] >= 0.5) | (spearman['SalePrice'] <= -0.5) ]

selected_features_corr = corred['index'].to_list()
selected_features_corr.remove(target)
print(len(selected_features_corr))
print(selected_features_corr)

In [ ]:
fig = plt.figure(figsize=(16,10) )
sns.heatmap( df_train[selected_features_corr].corr(),  annot=True   )
plt.title('Normal')
plt.show()

In [ ]:
cols_to_remove = ['fea_AllSF_log', 'fea_GrLivArea', 'fea_GrLivArea_log', 'fea_1stFlrSF', 'fea_FullBath',
                  'fea_GarageArea', 'fea_FireplaceQu', 'fea_ExterQual', 'fea_GarageYrBlt', 'fea_GarageCars', 'fea_TotalBsmtSF']
for cols in cols_to_remove:
    selected_features_corr.remove(cols)
print(len(selected_features_corr))

In [ ]:
fig = plt.figure(figsize=(16,10) )
sns.heatmap( df_train[selected_features_corr].corr(),  annot=True   )
plt.title('Normal')
plt.show()

In [ ]:
exp_vars = selected_features_corr.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
X = df_train[selected_features_corr]
y = df_train['SalePrice']

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break

selected_features_corr_BE = cols
print(selected_features_corr_BE)
print(len(selected_features_corr_BE))

In [ ]:
exp_vars = selected_features_corr_BE.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

vif_data = pd.DataFrame() 
vif_data["feature"] = df_train[features_with_correlation].columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df_train[features_with_correlation].values, i) 
                          for i in range(len(df_train[features_with_correlation].columns))]

# 5. REGRESSION WITH FEATURES DETERMINED BY f_regression

In [ ]:
X = df_train[fea_cols]
y = df_train['SalePrice']

fs = SelectKBest(score_func=f_regression, k='all')
fs.fit(X, y)
X_fs = fs.transform(X)

df_fs = pd.DataFrame()
df_fs['Feature'] = X.columns
df_fs['fs_score'] = fs.scores_
df_fs['fs_pvalue'] = fs.pvalues_

df_fs.sort_values('fs_score', ascending=False).head(30).plot( kind='bar', x='Feature', y='fs_score')
plt.show()

In [ ]:
k = 5
features_from_fs_score = df_fs.sort_values('fs_score', ascending=False).head(k)['Feature'].to_list()
# print(df_fs.sort_values('fs_score', ascending=False).head(k)['Feature'].to_list())

In [ ]:
exp_vars = features_from_fs_score.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
X = df_train[features_from_fs_score]
y = df_train['SalePrice']

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break

features_from_fs_score_BE = cols
print(features_from_fs_score_BE)
print(len(features_from_fs_score_BE))

In [ ]:
exp_vars = features_from_fs_score_BE.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

vif_data = pd.DataFrame() 
vif_data["feature"] = df_train[features_from_fs_score].columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df_train[features_from_fs_score].values, i) 
                          for i in range(len(df_train[features_from_fs_score].columns))]
vif_data

In [ ]:
fig = plt.figure(figsize=(12,8) )
sns.heatmap( df_train[features_from_fs_score_BE].corr(),  annot=True   )
plt.title('Normal')
plt.show()

# 6. REGRESSION WITH FEATURES DETERMINED BY Mutual Information

In [ ]:
X = df_train[fea_cols]
y = df_train['SalePrice']

mir = SelectKBest(score_func=mutual_info_regression, k='all')
mir.fit(X, y)
X_mir = mir.transform(X)

df_mir = pd.DataFrame()
df_mir['Feature'] = X.columns
df_mir['mir_score'] = mir.scores_
df_mir['mir_pvalue'] = mir.pvalues_

df_mir.sort_values('mir_score', ascending=False).head(30).plot( kind='bar', x='Feature', y='mir_score')
plt.show()

In [ ]:
k = 5
features_from_mir_score = df_mir.sort_values('mir_score', ascending=False).head(k)['Feature'].to_list()

In [ ]:
exp_vars = features_from_mir_score.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
X = df_train[features_from_mir_score]
y = df_train['SalePrice']

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break

features_from_mir_score_BE = cols
print(features_from_mir_score_BE)
print(len(features_from_mir_score_BE))

In [ ]:
exp_vars = features_from_mir_score_BE.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

##############################
##### SCIKIT-LEARN PART
##############################

reg          = LinearRegression()
reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

##############################
##### STATMODELS PART
##############################

X            = sm.add_constant(df_train[exp_vars])
y            = df_train['SalePrice']

model        = sm.OLS(y, X).fit() 
pred         = model.predict(X)

# Print out the statistics
print(model.summary())

##############################
##### RESIDUAL VISUALIZER
##############################

print("-"*25)
reg_result_visualizer(pred, y)

In [ ]:
fig = plt.figure(figsize=(12,8) )
sns.heatmap( df_train[features_from_mir_score_BE].corr(),  annot=True   )
plt.title('Normal')
plt.show()

vif_data = pd.DataFrame() 
vif_data["feature"] = df_train[features_from_mir_score].columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(df_train[features_from_mir_score].values, i) 
                          for i in range(len(df_train[features_from_mir_score].columns))]
vif_data

# 7. RIDGE REGRESSION

In [ ]:
exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

alpha_space = np.arange(0.1, 20, 0.1) # np.logspace(-12, -1, 400)

ridge = RidgeCV(alphas = alpha_space, store_cv_values = True)
ridge.fit(X, y)
cv_values = ridge.cv_values_.mean(axis=0)

alpha = ridge.alpha_
print("Best alpha :", alpha)



plt.plot(alpha_space, cv_values, label="ridge")

plt.xlabel("alpha")
plt.ylabel("mean squared error")
plt.title("RidgeCV Alpha Error")
plt.show()

In [ ]:
alpha_space = np.arange(0.1, 20, 0.1) # np.logspace(-12, -1, 400)
scores = []
model = Ridge()
for alpha in alpha_space:
    model.set_params(alpha=alpha)
    score = cross_val_score(model, X, y, cv=5)
    scores.append(score.mean())

res = pd.DataFrame({ 'alpha': alpha_space, 'score': scores })
print("Best alpha :", float(res[res['score'] == res['score'].max()]['alpha']))   

plt.plot(alpha_space, scores, label='ridge')
plt.xlabel('alpha')
plt.ylabel('R2')
plt.title("Manual Ridge Alpha Scoring")
plt.show()

In [ ]:
reg          = Ridge(alpha=alpha)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)
reg.fit(X_train, y_train)
y_pred       = reg.predict(X_test)
r2           = reg.score(X_test, y_test)
rmse         = np.sqrt(mean_squared_error(y_test, y_pred))

reg.fit(X, y)
y_pred_train = reg.predict(X)
r2_train     = reg.score(X, y)
rmse_train   = np.sqrt(mean_squared_error(y, y_pred_train))

cv_scores    = cross_val_score(reg, X, y, cv=5)
cv_score     = np.mean(cv_scores)

print(f"Train R^2:         {r2_train}  ")
print(f"Train RMSE:        {rmse_train}")
print(f"Test (0.3) R^2:    {r2}        ")
print(f"Test (0.3) RMSE:   {rmse}      ")
print(f"5-Fold Avg CV:     {cv_score}  ")
print(f"5-Fold CV Scores:  {cv_scores} ")

print('-'*25)
print(f'{sum(reg.coef_ == 0)} features eliminated!')
print(f'{sum(reg.coef_ != 0)} features utilized!')

# 8. LASSO REGRESSION

In [ ]:
exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

alpha_space = np.arange(-5, 5) # np.logspace(-12, -1, 400)

lasso = LassoCV(alphas = alpha_space)
lasso.fit(X, y)
cv_values = lasso.mse_path_.mean(axis=1)

alpha = lasso.alpha_
print("Best alpha :", alpha)



plt.plot(alpha_space, cv_values, label="lasso")

plt.xlabel("alpha")
plt.ylabel("mean squared error")
plt.title("LassoCV Alpha Error")
plt.show()

In [ ]:
alpha_space = np.arange(-5, 5, 0.1) # np.logspace(-12, -1, 400)
scores = []
model = Lasso()
for alpha in alpha_space:
    model.set_params(alpha=alpha)
    score = cross_val_score(model, X, y, cv=5, scoring='r2')
    scores.append(score.mean())

res = pd.DataFrame({ 'alpha': alpha_space, 'score': scores })
print("Best alpha :", float(res[res['score'] == res['score'].max()]['alpha']))   

plt.plot(alpha_space, scores, label='lasso')
plt.xlabel('alpha')
plt.ylabel('R2')
plt.title("Manual Ridge Alpha Scoring")
plt.show()

# 9. LASSOLARS REGRESSION

In [ ]:
exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values


lassoLars = LassoLarsCV()
lassoLars.fit(X, y)
cv_values = lassoLars.mse_path_.mean(axis=1)

alpha = lassoLars.alpha_
print("Best alpha :", alpha)



plt.plot(lassoLars.cv_alphas_, cv_values, label="lassoLars")

plt.xlabel("alpha")
plt.ylabel("mean squared error")
plt.title("LassoLarsCV Alpha Error")
plt.show()

# 10. ELASTICNET REGRESSION

In [ ]:
exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

alpha_space = np.arange(0, 5, 0.01) # np.logspace(-12, -1, 400)

ElasticNet = ElasticNetCV()
ElasticNet.fit(X, y)
cv_values = ElasticNet.mse_path_.mean(axis=1)

alpha = ElasticNet.alpha_
print("Best alpha :", alpha)



plt.plot(ElasticNet.alphas_, cv_values, label="ElasticNet")

plt.xlabel("alpha")
plt.ylabel("mean squared error")
plt.title("LassoLarsCV Alpha Error")
plt.show()

# 11. DECISION TREE REGRESSION

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE

exp_vars = fea_cols.copy()

X = df_train[exp_vars].values
y = df_train['SalePrice'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=28)

dt = DecisionTreeRegressor(max_depth=8,
                           min_samples_leaf=0.13,
                           random_state=28)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
mse_dt = MSE(y_test, y_pred)


print("Test set MSE of dt: {:.4f}".format(mse_dt))

In [ ]:
MSE_CV_scores = - cross_val_score(dt, X_train, y_train, cv=5, 
                                  scoring='neg_mean_squared_error', 
                                  n_jobs=-1) 
print(MSE_CV_scores.mean())
MSE_CV_scores

In [ ]:
dt.fit(X_train, y_train)
y_pred_train = dt.predict(X_train)
MSE(y_train, y_pred_train)

# 12. BAGGING REGRESSION

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

dt = DecisionTreeRegressor(max_depth=8,
                           min_samples_leaf=0.13,
                           random_state=28)
br = BaggingRegressor(base_estimator=dt, 
                       n_estimators=50,
                       oob_score=True,
                       random_state=28)


br.fit(X_train, y_train)
y_pred = br.predict(X_test)
mse_br = MSE(y_test, y_pred)

print("Test set MSE of dt: {:.4f}".format(mse_br))

# 13. RANDOM FOREST REGRESSION 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=25,
                           random_state=2)
        
rf.fit(X_train, y_train)                           
y_pred = rf.predict(X_test)

# Evaluate the test set RMSE
mse_rf = MSE(y_test, y_pred)

# Print rmse_test
print('Test set MSE of rf: {:.4f}'.format(mse_rf))


In [ ]:
importances = pd.DataFrame( {'feature': fea_cols,
                             'importance': rf.feature_importances_ } )
    
    
importances_sorted = importances.sort_values('importance', ascending=False)
importances_sorted.head(10)

In [ ]:
# Draw a horizontal barplot of importances_sorted
sns.barplot(x= importances_sorted['importance'].head(20),
            y= importances_sorted['feature'].head(20), orient='h',)
plt.title('Features Importances')
plt.show()

# 14. ADABOOST REGRESSION

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

dt = DecisionTreeRegressor(max_depth=4, random_state=28)
ada = AdaBoostRegressor(base_estimator=dt, n_estimators=100, random_state=1)

ada.fit(X_train, y_train)

y_pred = ada.predict(X_test)

mse_ada = MSE(y_test, y_pred)

print('Test set MSE of ada: {:.4f}'.format(mse_ada))

In [ ]:
importances = pd.DataFrame( {'feature': fea_cols,
                             'importance': ada.feature_importances_ } )
    
    
importances_sorted = importances.sort_values('importance', ascending=False)
importances_sorted.head(10)

# 15. GRADIENT BOOSTING REGRESSION

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as MSE

gb = GradientBoostingRegressor(max_depth=3,
                               n_estimators=200,
                               random_state=28)

gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)


mse_test = MSE(y_test, y_pred)
print('Test set MSE of gb: {:.3f}'.format(mse_test))

In [ ]:
importances = pd.DataFrame( {'feature': fea_cols,
                             'importance': ada.feature_importances_ } )
    
    
importances_sorted = importances.sort_values('importance', ascending=False)
importances_sorted.head(10)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as MSE

sgbr = GradientBoostingRegressor(max_depth=3, 
                                 subsample=0.8,
                                 max_features=0.75,
                                 n_estimators=200,                                
                                 random_state=28)

sgbr.fit(X_train, y_train)
y_pred = sgbr.predict(X_test)


mse_test = MSE(y_test, y_pred)
print('Test set MSE of gb: {:.3f}'.format(mse_test))

In [ ]:
importances = pd.DataFrame( {'feature': fea_cols,
                             'importance': ada.feature_importances_ } )
    
    
importances_sorted = importances.sort_values('importance', ascending=False)
importances_sorted.head(10)

# ................TO BE CONTINUED..............

In [ ]:
train['id'] = train['item_id'] + '_' + train['store_id']
print(train.shape)
print('-'*25)
train.head()

In [ ]:
# Making data usable in models
with timer():
    train = pd.melt( train,
                     id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                     value_vars = [col for col in train.columns if col.startswith("d_")],
                     var_name = "d",
                     value_name = "sales")
with timer():
    train['d']     = train['d'].str.extract('(\d+)').astype('int16')
    train['sales'] = train['sales'].astype('int16')


with timer():
    train.sort_values(['id', 'd'], inplace=True)


print(train.shape)
print('-'*25)
train.tail()

In [ ]:
# Adding the 28 days we try to forecast
with timer():
    tmp = train.groupby('id', as_index=False ).tail(28)
    tmp['d'] = tmp['d'] + 28
    tmp['sales'] = np.nan
    print(tmp.shape)

with timer():
    train = pd.concat([train,tmp], axis=0, sort=False, ignore_index=True)

del tmp; gc.collect()

with timer():
    train.sort_values(['id', 'd'], inplace=True)
    train.reset_index(drop=True, inplace=True)
    
print(train.shape)
print('-'*25)
train.tail()

In [ ]:
## Adding lag days
def lagger(train, lag_day):
    train[f'sales_lag_{lag_day}'] = train[['id', 'd', 'sales']].groupby('id')['sales'].transform( lambda x: x.shift(lag_day) ).astype('float16')
    return train

In [ ]:
for i in [1,2,3,4,5,6,7,8,12,13,14,15,28]:
    train = lagger(train,i)
    print(i, 'DONE')

In [ ]:
print(train.shape)
print('-'*25)
train.tail()

### 2.2. CALENDAR DATA PREPARATION

In [ ]:
print(calendar.shape)
print('-'*25)
calendar.tail()

In [ ]:
with timer():
    calendar['d'] = calendar['d'].str.extract('(\d+)').astype('int16')
    
    calendar['day'] = pd.to_datetime(calendar['date']).dt.day
    
    for col in ['wday', 'month', 'wm_yr_wk', 'year']:
        calendar[col] = calendar[col].astype('int16')
    
    calendar.drop(['date', 'weekday'], axis=1, inplace=True)
        
print(calendar.shape)
print('-'*25)
calendar.tail()

In [ ]:
## DO NOT FORGET TO ADD SNAP DATA
snap = calendar[['d', 'snap_CA', 'snap_TX', 'snap_WI']]

snap = pd.melt( snap,
                     id_vars = ['d'],
                     value_vars = [col for col in snap.columns if col.startswith("snap_")],
                     var_name = "state_id",
                     value_name = "is_snap")

snap['state_id'] = snap['state_id'].str[5:].astype('category')
snap['is_snap'] = snap['is_snap'].astype('int8')

print(snap.shape)
print('-'*25)
snap.tail()

In [ ]:
calendar.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis=1, inplace=True)

In [ ]:
tmp = pd.get_dummies(calendar['event_name_1'], dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

tmp = pd.get_dummies(calendar['event_type_1'], dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)


tmp = pd.get_dummies(calendar['event_name_2'], prefix = '2', dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

tmp = pd.get_dummies(calendar['event_type_2'], prefix = '2', dtype ='int8' )
calendar = pd.concat([calendar, tmp], axis=1)

calendar.drop(['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis=1, inplace=True)
del tmp; gc.collect()

print(calendar.shape)
print('-'*25)
calendar.tail(5)

In [ ]:
calendar['Cinco De Mayo']    = calendar['Cinco De Mayo']   + calendar['2_Cinco De Mayo']
calendar['Easter']           = calendar['Easter']          + calendar['2_Easter']
calendar['Father\'s day']    = calendar['Father\'s day']   + calendar['2_Father\'s day']
calendar['OrthodoxEaster']   = calendar['OrthodoxEaster']  + calendar['2_OrthodoxEaster']
calendar['Cultural']         = calendar['Cultural']        + calendar['2_Cultural']
calendar['Religious']        = calendar['Religious']       + calendar['2_Religious']

calendar.drop([col for col in calendar.columns if col.startswith('2_')], axis=1, inplace=True)

print(calendar.shape)
print('-'*25)
calendar.tail(5)

In [ ]:
calendar.loc[ (calendar['d'] >= 185 )  & ( calendar['d'] < 185 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 539 )  & ( calendar['d'] < 539 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 893 )  & ( calendar['d'] < 893 + 29 ), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1248)  & ( calendar['d'] < 1248 + 29), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1602)  & ( calendar['d'] < 1602 + 29), 'Ramadan starts' ]  = 1
calendar.loc[ (calendar['d'] >= 1957)                                , 'Ramadan starts' ]  = 1

calendar.loc[ (calendar['d'] >= 185 )  & ( calendar['d'] < 185 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 539 )  & ( calendar['d'] < 539 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 893 )  & ( calendar['d'] < 893 + 29 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1248)  & ( calendar['d'] < 1248 + 29), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1602)  & ( calendar['d'] < 1602 + 29), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1957)                                , 'Religious' ]  = 1


In [ ]:
calendar.loc[ (calendar['d'] >= 215 - 1  )  & ( calendar['d'] <= 215 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 569 - 1  )  & ( calendar['d'] <= 569 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 923 - 1  )  & ( calendar['d'] <= 923 + 1  ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 1278 - 1 )  & ( calendar['d'] <= 1278 + 1 ), 'Eid al-Fitr' ]  = 1
calendar.loc[ (calendar['d'] >= 1632 - 1 )  & ( calendar['d'] <= 1632 + 1 ), 'Eid al-Fitr' ]  = 1

calendar.loc[ (calendar['d'] >= 215 - 1  )  & ( calendar['d'] <= 215 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 569 - 1  )  & ( calendar['d'] <= 569 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 923 - 1  )  & ( calendar['d'] <= 923 + 1  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1278 - 1 )  & ( calendar['d'] <= 1278 + 1 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1632 - 1 )  & ( calendar['d'] <= 1632 + 1 ), 'Religious' ]  = 1

In [ ]:
calendar.loc[ (calendar['d'] >= 283   )  & ( calendar['d'] <= 283 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 637   )  & ( calendar['d'] <= 637 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 991   )  & ( calendar['d'] <= 991 + 3  ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 1345  )  & ( calendar['d'] <= 1345 + 3 ), 'EidAlAdha' ]  = 1
calendar.loc[ (calendar['d'] >= 1700  )  & ( calendar['d'] <= 1700 + 3 ), 'EidAlAdha' ]  = 1

calendar.loc[ (calendar['d'] >= 283   )  & ( calendar['d'] <= 283 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 637   )  & ( calendar['d'] <= 637 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 991   )  & ( calendar['d'] <= 991 + 3  ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1345  )  & ( calendar['d'] <= 1345 + 3 ), 'Religious' ]  = 1
calendar.loc[ (calendar['d'] >= 1700  )  & ( calendar['d'] <= 1700 + 3 ), 'Religious' ]  = 1

In [ ]:
calendar.loc[ (calendar['d'] >= 123  )  & ( calendar['d'] <= 135  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 501  )  & ( calendar['d'] <= 510  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 860  )  & ( calendar['d'] <= 874  ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1224 )  & ( calendar['d'] <= 1234 ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1588 )  & ( calendar['d'] <= 1600 ), 'NBAFinalsStart' ]  = 1
calendar.loc[ (calendar['d'] >= 1952 )  & ( calendar['d'] <= 1969 ), 'NBAFinalsStart' ]  = 1

calendar.loc[ (calendar['d'] >= 123  )  & ( calendar['d'] <= 135  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 501  )  & ( calendar['d'] <= 510  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 860  )  & ( calendar['d'] <= 874  ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1224 )  & ( calendar['d'] <= 1234 ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1588 )  & ( calendar['d'] <= 1600 ), 'Sporting' ]  = 1
calendar.loc[ (calendar['d'] >= 1952 )  & ( calendar['d'] <= 1969 ), 'Sporting' ]  = 1

calendar.drop(['NBAFinalsEnd'], axis=1, inplace=True)

In [ ]:
# Merging calendar 
with timer():
    train = train.merge(calendar, on='d', how='left')

print(train.shape)
print('-'*25)
train.tail()

In [ ]:
del calendar; gc.collect()

In [ ]:
train.info()

### 2.3. PRICE DATA PREPARATION

In [ ]:
print(prices.shape)
print('-'*25)
prices.head()

In [ ]:
prices['sell_price_over_mean']   = (prices['sell_price'] / prices.groupby(['store_id', 'item_id'])['sell_price'].transform('mean')   ) - 1
prices['sell_price_over_median'] = (prices['sell_price'] / prices.groupby(['store_id', 'item_id'])['sell_price'].transform('median') ) - 1

prices['sell_price_over_mean']   = prices['sell_price_over_mean'  ].astype('float16')
prices['sell_price_over_median'] = prices['sell_price_over_median'].astype('float16')

print(prices.shape)
print('-'*25)
prices.head()

In [ ]:
# Merging prices 
with timer():
    train = train.merge(prices, on=['item_id','store_id','wm_yr_wk'], how='inner')

print(train.shape)
print('-'*25)
train.tail()

In [ ]:
del prices; gc.collect()

In [ ]:
for col in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']:
    train[col] = train[col].astype('category')

## 3 FEATURE ENGINEERING

In [ ]:
## DO NOT FORGET TO ADD SNAP DATA
with timer():
    train = train.merge(snap, on=['d', 'state_id'], how='left')

print(train.shape)
print('-'*25)
train.tail()

In [ ]:
del snap; gc.collect()

### Feature 0  >>>  id based nonzero statistics

In [ ]:
## With this id is obsolete
f0 = train[train['sales'] > 0 ].groupby('id').agg({ 'sales': ['mean', 'std'] }).reset_index()
f0.columns = [ f[0] if f[1] == '' else 'id_nonzero_' + f[0] + '_' + f[1] for f in f0.columns ]

print(f0.shape)
print('-'*25)
f0.tail()

In [ ]:
# Merging f0
with timer():
    train = train.merge(f0, on='id', how='left')

del f0; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

### Feature 1  >>>  id based statistics

In [ ]:
## With this id is obsolete
f1 = train.groupby('id').agg({ 'sales': ['mean', 'std'] }).reset_index()
f1.columns = [ f[0] if f[1] == '' else 'id_' + f[0] + '_' + f[1] for f in f1.columns ]

print(f1.shape)
print('-'*25)
f1.tail()

In [ ]:
# Merging f1
with timer():
    train = train.merge(f1, on='id', how='left')

del f1; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

### Feature 2  >>>  item_id based statistics

In [ ]:
## With this item_id is obsolete
f2 = train.groupby('item_id').agg({ 'sales': ['mean', 'std'] }).reset_index()
f2.columns = [ f[0] if f[1] == '' else 'item_id_' + f[0] + '_' + f[1] for f in f2.columns ]

print(f2.shape)
print('-'*25)
f2.tail()

In [ ]:
# Merging f2
with timer():
    train = train.merge(f2, on='item_id', how='left')

del f2; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

### Feature 3  >>>  dept_id based statistics

In [ ]:
## With this dept_id is obsolete
f3 = train.groupby('dept_id').agg({ 'sales': ['mean', 'std'] }).reset_index()
f3.columns = [ f[0] if f[1] == '' else 'dept_id_' + f[0] + '_' + f[1] for f in f3.columns ]

print(f3.shape)
print('-'*25)
f3.tail()

In [ ]:
# Merging f3
with timer():
    train = train.merge(f3, on='dept_id', how='left')

del f3; gc.collect()

print(train.shape)
print('-'*25)
train.tail()

In [ ]:
store_list = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

for store in store_list:
    with timer():
        print(store)
        train[train['store_id'] == store].to_csv(f'{path}FE3/Train_Data_2_{store}.csv', index=False)

# STORE BASED FEATURE ENGINEERING AND MODELLING

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', -1)

import seaborn as sns
sns.set_style('whitegrid')

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


from sklearn import feature_selection
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

print('** M5 - FORECASTING - ACCURACY **')
print("-"*25)

import gc
import os
for x in os.listdir("input/"):
    print(x)
print("-"*25)

path = 'input/'
out_path = 'output/FE3/'

print ("Initiation is DONE !!")
print("-"*25)

In [ ]:
##### SOME USEFUL FUNCTIONS

## TIMER
import time
from contextlib import contextmanager
@contextmanager
def timer():
    t0 = time.time()
    yield
    t1 = time.time()
    print( 'Runtime: ' + str(round(t1-t0,2)) + ' sn' )

### Store Feature 4 >>> id-monthdaily based statistics

In [ ]:
def feature4(train):
    f4 = train.groupby(['id', 'day']).agg({ 'sales': ['mean', 'std'] }).reset_index()
    f4.columns = [ f[0] if f[1] == '' else 'id_day_' +  f[0] + '_' + f[1] for f in f4.columns ]

    with timer():
        train = train.merge(f4, on=['id', 'day'], how='left')

    del f4; gc.collect()
    
    return train

### Store Feature 5 >>> id-weekdaily based statistics

In [ ]:
def feature5(train):
    f5 = train.groupby(['id', 'wday']).agg({ 'sales': ['mean', 'std'] }).reset_index()
    f5.columns = [ f[0] if f[1] == '' else 'id_wday_' +  f[0] + '_' + f[1] for f in f5.columns ]
    
    with timer():
        train = train.merge(f5, on=['id', 'wday'], how='left')

    del f5; gc.collect()
    
    return train

### Store Feature 6 >>> id-monthly based statistics

In [ ]:
def feature6(train):
    f6 = train.groupby(['id', 'month']).agg({ 'sales': ['mean', 'std'] }).reset_index()
    f6.columns = [ f[0] if f[1] == '' else 'id_month_' + f[0] + '_' + f[1] for f in f6.columns ]

    with timer():
        train = train.merge(f6, on=['id', 'month'], how='left')

    del f6; gc.collect()
    
    return train

### Store Feature 7 >>> event based statistics

In [ ]:
def feature7(train):
    
    event_cat = ['Cultural', 'National', 'Religious', 'Sporting']

    for event in event_cat:

        with timer():
            f7 = train[train[f'{event}'] > 0 ].groupby(['id']).agg({ 'sales': ['mean', 'std'] }).reset_index()
            f7.columns = [ f[0] if f[1] == '' else f'id_{event}_' + f[0] + '_' + f[1] for f in f7.columns ]
            f7[f'{event}'] = 1

            train = train.merge(f7, on=['id', f'{event}'], how='left')
            new_cols = [col for col in train.columns if col.startswith(f'id_{event}_')]

            for c in new_cols:
                train[c] = train[c].fillna(0)

            del f7; gc.collect()

    return train

### Prediction Function

In [ ]:
def predictor(model, store, pred, y_true, param_name, submission, feature_columns_to_keep):

    print('-'*25)
    print( 'MODEL PREDICTION BEGAN ...' )
    print('-'*25)
    
    
    y_pred = model.predict(pred[feature_columns_to_keep])
    x_pred = pred.copy()

    x_pred['y_true'] = y_true
    x_pred['y_pred'] = y_pred
    x_pred['y_pred'] = x_pred['y_pred'].apply(lambda x: 0 if x<0 else x)
    
    
    sns.lineplot(x='d', y='y_true', data=x_pred[ ( x_pred['id'] == f'FOODS_3_013_{store}' ) ] , color='r')
    sns.lineplot(x='d', y='y_pred', data=x_pred[ ( x_pred['id'] == f'FOODS_3_013_{store}' ) ] , color='b')
    plt.show()
    
    sns.lineplot(x='d', y='y_true', data=x_pred[ ( x_pred['id'] == f'HOUSEHOLD_2_102_{store}' ) ], color='r')
    sns.lineplot(x='d', y='y_pred', data=x_pred[ ( x_pred['id'] == f'HOUSEHOLD_2_102_{store}' ) ], color='b')
    plt.show()    

    
    res_val  = x_pred[x_pred['d'] <= 1941][['id', 'd', 'y_pred']].copy()
    res_eval = x_pred[x_pred['d'] > 1941][['id', 'd', 'y_pred']].copy() 

    res_val['id']  = res_val['id'].astype(str)  + '_validation'
    res_eval['id'] = res_eval['id'].astype(str)  + '_evaluation'

    res_val             = res_val.pivot_table(index='id', columns='d', values='y_pred').reset_index()
    res_eval            = res_eval.pivot_table(index='id', columns='d', values='y_pred').reset_index()
    res_eval.columns    = res_val.columns

    res            = pd.concat([res_val, res_eval], axis=0, sort=False, ignore_index=True)
    res            = submission[['id']].merge(res, on='id', how='inner')
    res.columns    = submission.columns

    res.to_csv(f'{out_path}model_lgbm_2_{store}_' + param_name + '.csv', index=False)

    
    del x_pred, y_pred, res_val, res_eval, res; gc.collect()
    
    print('-'*25)
    print( 'MODEL PREDICTION COMPLETED !!!' )
    print('-'*25)

## The Real Deal

In [ ]:
submission = pd.read_csv( f'{path}sample_submission.csv')

store_list = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

for store in store_list:
    train = pd.read_csv(f'{path}FE3/Train_Data_2_{store}.csv')
    
    train = feature4(train)
    train = feature5(train)
    train = feature6(train)
    train = feature7(train)
    
    cols_to_drop = ['item_id', 'store_id', 'state_id', 'wm_yr_wk']
    event_cat = ['Cultural', 'National', 'Religious', 'Sporting']
    
    train.drop(cols_to_drop, axis=1, inplace=True)
    train.drop(event_cat, axis=1, inplace=True)
    
    cat_cols = ['id', 'dept_id', 'cat_id' ]
    for col in cat_cols:
        train[col] = train[col].astype('category') 

    pred = train[train['d'] > 1913]
    y_true = pred['sales']
    pred = pred.drop('sales', axis=1)

    valid = train[ (train['d'] <= 1941) & (train['d'] > 1913) ]
    y_valid = valid['sales']
    x_valid = valid.drop('sales', axis=1)

    x_train = train.drop('sales', axis=1)
    y_train = train['sales']

    feature_columns_to_keep = [col for col in x_train.columns if col != 'd' ] # if ((col != 'id') & (col != 'd')) ]

    train_data = lgb.Dataset( x_train[feature_columns_to_keep], categorical_feature=cat_cols, label = y_train, free_raw_data=False )
    valid_data = lgb.Dataset( x_valid[feature_columns_to_keep], categorical_feature=cat_cols, label = y_valid, free_raw_data=False )

    del valid, x_train, x_valid, y_train, y_valid ; gc.collect()
    
    
    res_rmse = pd.DataFrame()
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = feature_columns_to_keep


    obj_list   = [ 'regression', 'poisson']   # , 'tweedie' 
    boost_list = [ 'gbdt']
    lr_list    = [ 0.7, 0.5, 0.3 ]
    l1_list    = [ 0 ]
    l2_list    = [ 0 ]

    i=0


    for obj in obj_list:
        for boost in boost_list:
            for lr in lr_list: 
                for l1 in l1_list:
                    for l2 in l2_list:
                        i += 1

                        params = {
                        # 'regression', 'regression_l1', 'tweedie',  'poisson', 'quantile', 'gamma', 'multiclass', 'cross_entropy'
                        'objective'         : obj        ,

                        #
                        # 'num_class'         : len(set(train_data.label)) ,

                        # 'gbdt', 'rf', 'dart', 'goss'
                        'boosting'          : boost      ,

                        # number of boosting iterations (100)  - [0, )
                        'num_iterations'    : 6000        ,    

                        # shrinkage rate (0.1) - [0, )
                        'learning_rate'     : lr         ,

                        # 'serial', 'feature', 'data', 'voting'
                        'tree_learner'      : 'serial'   ,

                        # if there are too many rows make it True, if there are too many columns make 'force_col_wise' option True
                        # Never make them both True
                        'force_row_wise'    : True       ,

                        # if the data is small use it otherwise don't touch it
                        'max_depth'         : -1         ,

                        # 20 - [0, )
                        'min_data_in_leaf'  : 31         , 

                        # randomly select part of data without resampling, to enable bagging it must be smaller than 1 (0,1]
                        # 'bagging_fraction'  : 1          ,

                        # to enable bagging bagging_freq must be nonzero [0, )
                        #'bagging_freq'      : 0          ,

                        # randomly select features, to enable featuring it must be smaller than 1 (0,1]
                        # 'feature_fraction'  : 1          ,

                        # stops training if one metric of one validation data does not improve in the last x rounds (0)
                        'early_stopping_round' : 1000     ,

                        # (0)
                        'lambda_l1'         : l1         ,

                        # (0)
                        'lambda_l2'         : l2          ,

                        #controls the level of LGBM verbosity
                        'verbosity'         : 1          ,  

                        # 'rmse', 'auc', 'l1', 'l2', 'tweedie', 'poissson', 'multi_logloss'..
                        'metric'            : 'rmse'      ,

                        # Use these 3 parameters to handle memory error
                        # -----------------------------------------------------
                        # max number of leaves in one tree (31) - [1,131072]
                        'num_leaves'              : 128    , 
                        # max cache size in MB for historical histogram (-1)
                        #'histogram_pool_size'     : 128   ,
                        # max number of bins that feature values will be bucketed in (255) - 
                        #'max_bin'                 : 31    ,
                        # -----------------------------------------------------

                        'random_state': 28,

                        }

                        with timer():
                            model = lgb.train(params, train_data, 1000, valid_sets = [valid_data], categorical_feature=cat_cols, verbose_eval=100 ) 
                            mtrc_rmse = model.best_score['valid_0']['rmse']
                            param_name = f'{i}_{obj}_{boost}_{str(lr)}_{str(l1)}_{str(l2)}_{str(mtrc_rmse)}'


                        model.save_model(f'{out_path}model_lgbm_2_{store}_' + param_name + '.txt')

                        res_rmse = res_rmse.append( pd.DataFrame( {  'i': [i],
                                                 'objective': [obj],
                                                 'boosting': [boost],
                                                 'learning_rate':[lr], 
                                                 'l1':[l1], 
                                                 'l2':[l2], 
                                                 'RMSE': [mtrc_rmse]  } ) )

                        feature_importances[param_name] = model.feature_importance()               

                        plt.figure( figsize=(12,8) )
                        sns.barplot( x    = param_name,
                                     y    = 'feature',
                                     data = feature_importances.sort_values(param_name, ascending = False ) )
                        plt.title('LGBM Feature Importance ' + store + ' ' + param_name)
                        plt.show()

                        predictor(model, store, pred, y_true, param_name, submission, feature_columns_to_keep)

                        print('-'*25)
                        print( res_rmse )
                        print('-'*25)


    res_rmse.to_excel(f'{out_path}res_rmse_{store}.xlsx', index=False)                 
    feature_importances.to_excel(f'{out_path}feature_importances_{store}.xlsx', index=False) 


In [ ]:
for x in os.listdir("output/"):
    if x.

In [ ]:
matching = [e for e in os.listdir("output/") if (("model_lgbm_2" in e ) & ("regression_gbdt_0.5" in e ) & (".csv" in e ) ) ]
matching

In [ ]:
df = pd.DataFrame()
for doc in matching:
    df = df.append( pd.read_csv('output/' + doc) )

print(df.shape)
df.head()

In [ ]:
df = df[ df['F1'].notnull() ]
print(df.shape)
df.head()

In [ ]:
col_list = [col for col in df.columns if col.startswith("F")]

In [ ]:
for col in col_list:
    df[col] = df[col].apply(lambda x: 0 if x<0 else x)

In [ ]:
df.to_csv('model_lgbm_2_1_regression_gbdt_0.5_0_0.csv', index=False)

In [ ]:
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [ ]:
#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

In [ ]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': 1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons